# Import library

In [1]:
import os
import os, sys
import glob
import re
import datetime
from datetime import datetime, timedelta 
import math, copy, time
from itertools import cycle
from collections import defaultdict

from parse import parse
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

# import torchvision
# import torchvision.transforms as transforms

from sklearn.metrics import roc_auc_score, plot_roc_curve
from sklearn import svm


from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn

%load_ext autoreload
%autoreload 2

In [2]:
# Util functions
def size_and_ratio(df, col, dropna=True):
    if pd.__version__ > '1.1':
        sr = df.groupby(col, dropna=dropna).size().sort_values(ascending=False)
    elif dropna:
        sr = df.groupby(col).size().sort_values(ascending=False)
    else:  # dropna == False
        sr = df.fillna('nan').groupby(col).size().sort_values(ascending=False)
    sr_ratio = sr.copy() / sum(sr)
    print("Sum :", sum(sr), sr.shape)
    sr.name = 'size'
    sr_ratio.name = 'ratio'
    return pd.concat([sr, sr_ratio], axis=1)

# Data load

In [3]:
df_input = pd.read_csv("opendataset/df_input.csv")
df_input.head()

,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender,confirmed_ratio,weekday_ratio,corona_result
0,0,0,0,0,0,-1,0,0.040948,0.0,0
1,0,0,0,0,0,-1,1,0.040948,0.0,0
2,0,0,0,0,0,-1,0,0.040948,0.0,0
3,0,0,0,0,0,-1,0,0.040948,0.0,0
4,0,0,0,0,0,-1,1,0.040948,0.0,0


In [4]:
from model.attention_model import CategoricalAttentionModel
from model.utils import get_dict_category_from_dataset
from model.utils import get_category_key

In [5]:
dict_category, df_input_typed = get_dict_category_from_dataset(df_input)
dict_category

cough category [0 1] ...
fever category [0 1] ...
sore_throat category [0 1] ...
shortness_of_breath category [0 1] ...
head_ache category [0 1] ...
age_60_and_above category [-1  0  1] ...
gender category [0 1] ...
confirmed_ratio float [0.04094774 0.05944027 0.07859325 0.06852142 0.07132832 0.10071824
 0.12309089 0.09229753 0.11838521 0.11731198]
weekday_ratio category [0.         0.33333333 0.5        0.66666667 0.83333333 0.16666667
 1.        ] ...
corona_result category [0 1] ...


{'cough_0': 0,
 'cough_1': 1,
 'fever_0': 2,
 'fever_1': 3,
 'sore_throat_0': 4,
 'sore_throat_1': 5,
 'shortness_of_breath_0': 6,
 'shortness_of_breath_1': 7,
 'head_ache_0': 8,
 'head_ache_1': 9,
 'age_60_and_above_-1': 10,
 'age_60_and_above_0': 11,
 'age_60_and_above_1': 12,
 'gender_0': 13,
 'gender_1': 14,
 'confirmed_ratio': 15,
 'weekday_ratio_0': 16,
 'weekday_ratio_1': 17,
 'corona_result_0': 18,
 'corona_result_1': 19}

### category 값, intensity 값 생성

In [6]:
df_train = df_input_typed[[x for x in df_input_typed.columns if x != 'corona_result']]

df_train_category = pd.DataFrame()
df_train_intensity = df_train_category.copy()

for index, row in df_train.iterrows():
    for col, val in row.items():
        type_name = df_train[col].dtype
        key = get_category_key(colname=col, type_name=type_name, value=val)
        cat_idx = dict_category[key]

        df_train_category.loc[index, col] = cat_idx
        if type_name == 'category':
            df_train_intensity.loc[index, col] = 1
        if type_name == 'float':
            df_train_intensity.loc[index, col] = val


display(df_train_category.sample(10))
display(df_train_intensity.sample(10))

,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender,confirmed_ratio,weekday_ratio
139,0.0,2.0,4.0,6.0,8.0,10.0,14.0,15.0,16.0
5285,0.0,2.0,4.0,6.0,8.0,12.0,13.0,15.0,16.0
1226,0.0,2.0,4.0,6.0,8.0,11.0,14.0,15.0,17.0
3844,0.0,2.0,4.0,6.0,8.0,11.0,13.0,15.0,16.0
1631,0.0,2.0,4.0,6.0,8.0,11.0,13.0,15.0,16.0
1108,0.0,2.0,4.0,6.0,8.0,11.0,13.0,15.0,16.0
2770,0.0,2.0,4.0,6.0,8.0,11.0,14.0,15.0,16.0
5430,0.0,2.0,4.0,6.0,8.0,11.0,14.0,15.0,16.0
6101,0.0,2.0,4.0,6.0,8.0,12.0,14.0,15.0,16.0
2857,0.0,2.0,4.0,6.0,8.0,11.0,13.0,15.0,16.0


,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender,confirmed_ratio,weekday_ratio
3684,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.343763,1.0
1667,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.181210,1.0
5466,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.600760,1.0
1716,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.192190,1.0
6521,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.524395,1.0
1139,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.117312,1.0
755,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.123091,1.0
5474,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.600760,1.0
1628,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.181210,1.0
592,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.100718,1.0


예시로 sore_throat id값 확인해봄

In [7]:
size_and_ratio(df_train_category, 'sore_throat')

Sum : 7230 (2,)


,size,ratio
sore_throat,,
4.0,7173,0.992116
5.0,57,0.007884


In [8]:
def split_train_test(data, test_ratio, seed=None):
    if seed != None:
        np.random.seed(seed)
    
    shuffled_indices = np.random.permutation(len(data)).astype(int)
    print("shuffled:", shuffled_indices[:5], '...', "total len:", len(data))
    test_set_size = int(len(data) * test_ratio)
    
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    
    test_data = data[:test_set_size]
    train_data = data[test_set_size:]
    test_data = data[test_indices]
    train_data = data[train_indices]
    
    return train_data, test_data
def split_input_gt(non_split_data):
    splited_input = non_split_data[:, :-1]
    splited_gt = non_split_data[:, -1]
    
    return splited_input, splited_gt

In [9]:
np_cat = df_train_category.values
np_ins = df_train_intensity.values
np_gt = df_input_typed[['corona_result']].values
np_gt

array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]])

In [10]:
not_test_np_cat, test_np_cat = split_train_test(np_cat, test_ratio=0.2, seed=1212)
train_np_cat, valid_np_cat = split_train_test(not_test_np_cat, test_ratio=0.2, seed=1212)

not_test_np_ins, test_np_ins = split_train_test(np_ins, test_ratio=0.2, seed=1212)
train_np_ins, valid_np_ins = split_train_test(not_test_np_ins, test_ratio=0.2, seed=1212)


not_test_np_gt, test_np_gt = split_train_test(np_gt, test_ratio=0.2, seed=1212)
train_np_gt, valid_np_gt = split_train_test(not_test_np_gt, test_ratio=0.2, seed=1212)


shuffled: [3571  241 1807 3269 2200] ... total len: 7230
shuffled: [2451 4535 4270 4511 1652] ... total len: 5784
shuffled: [3571  241 1807 3269 2200] ... total len: 7230
shuffled: [2451 4535 4270 4511 1652] ... total len: 5784
shuffled: [3571  241 1807 3269 2200] ... total len: 7230
shuffled: [2451 4535 4270 4511 1652] ... total len: 5784


In [11]:
train_input = [train_np_cat.copy(), train_np_ins.copy()]
train_gt = train_np_gt.copy()

valid_input = [valid_np_cat.copy(), valid_np_ins.copy()]
valid_gt = valid_np_gt.copy()

test_input = [test_np_cat.copy(), test_np_ins.copy()]
test_gt = test_np_gt.copy()


In [12]:
print(train_input[0].shape, valid_input[0].shape, test_input[0].shape)
print(train_gt.shape, valid_gt.shape, test_gt.shape)

(4628, 9) (1156, 9) (1446, 9)
(4628, 1) (1156, 1) (1446, 1)


In [13]:
print("# of train data positive: ", train_gt.sum())
print("# of train data negative: ", train_gt.shape[0] - train_gt.sum())
print("ratio of P/N: ", train_gt.sum()/(train_gt.shape[0] - train_gt.sum()))
print("")

print("# of valid data positive: ", valid_gt.sum())
print("# of valid data negative: ", valid_gt.shape[0] - valid_gt.sum())
print("ratio of P/N: ", valid_gt.sum()/(valid_gt.shape[0] - valid_gt.sum()))
print("")

print("# of test data positive: ", test_gt.sum())
print("# of test data negative: ", test_gt.shape[0] - test_gt.sum())
print("ratio of P/N: ", test_gt.sum()/(test_gt.shape[0] - test_gt.sum()))

# of train data positive:  335
# of train data negative:  4293
ratio of P/N:  0.07803400885161892

# of valid data positive:  90
# of valid data negative:  1066
ratio of P/N:  0.08442776735459662

# of test data positive:  98
# of test data negative:  1348
ratio of P/N:  0.07270029673590504


### Check dict_category

In [14]:
dict_category

{'cough_0': 0,
 'cough_1': 1,
 'fever_0': 2,
 'fever_1': 3,
 'sore_throat_0': 4,
 'sore_throat_1': 5,
 'shortness_of_breath_0': 6,
 'shortness_of_breath_1': 7,
 'head_ache_0': 8,
 'head_ache_1': 9,
 'age_60_and_above_-1': 10,
 'age_60_and_above_0': 11,
 'age_60_and_above_1': 12,
 'gender_0': 13,
 'gender_1': 14,
 'confirmed_ratio': 15,
 'weekday_ratio_0': 16,
 'weekday_ratio_1': 17,
 'corona_result_0': 18,
 'corona_result_1': 19}

## Let's Train imported class (simple attention module without inesity encoding)

In [15]:
from model.attention_model import CategoricalAttentionModel,CategoricalAttentionSimpleModel
# from model.attention_model import DataIteratorForIsraelModel

In [16]:
# from model.attention_model import IsraelDataset

In [17]:
# train_data_iter = IsraelDataset(train_input[0], train_input[1], train_gt, batch_size=64, shuffle=True)
# batch = next(iter(train_data_iter))

In [18]:
train_input

[array([[ 0.,  2.,  4., ..., 13., 15., 16.],
        [ 0.,  2.,  4., ..., 13., 15., 16.],
        [ 0.,  2.,  4., ..., 14., 15., 16.],
        ...,
        [ 0.,  2.,  4., ..., 13., 15., 16.],
        [ 0.,  2.,  4., ..., 14., 15., 16.],
        [ 0.,  2.,  4., ..., 13., 15., 17.]]),
 array([[1.        , 1.        , 1.        , ..., 1.        , 0.3437629 ,
         1.        ],
        [1.        , 1.        , 1.        , ..., 1.        , 0.86237926,
         1.        ],
        [1.        , 1.        , 1.        , ..., 1.        , 0.07132832,
         1.        ],
        ...,
        [1.        , 1.        , 1.        , ..., 1.        , 0.70882523,
         1.        ],
        [1.        , 1.        , 1.        , ..., 1.        , 0.48897878,
         1.        ],
        [1.        , 1.        , 1.        , ..., 1.        , 0.10352514,
         1.        ]])]

In [19]:
print("cat value:", train_input[0].shape)
print("intensity value:", train_input[1].shape)
print("Ground truth value:", train_gt.shape)

cat value: (4628, 9)
intensity value: (4628, 9)
Ground truth value: (4628, 1)


In [20]:
def validation(model, metrics, val_input, val_gt):
    
    # ************************
    val_pred, attn_val = model(val_input)
    # ************************
    print(val_pred.shape, val_gt.shape)
    result = dict()
    for key, metric_func in metrics.items():
        result[key] = metric_func(val_pred.detach(), val_gt.detach())
    return result

## CategoricalAttentionSimpleModel
무작위로 뽑아서 1000step 학습

In [21]:
d_model = 16
n_head = 4
model = CategoricalAttentionSimpleModel(dict_category, d_model=d_model, n_head=n_head, len_seq=train_input[0].shape[1])
loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
batch_size = 64

n_steps = 20000
log_result = dict()

model.train()
for cur_step in range(n_steps):
    """Train on single item of train data"""
    start = time.time()
    total_tokens = 0
    total_loss = 0

    model.train()
    
    train_batch_cat_value , train_batch_intensity_value = train_input
    random_indeicies = torch.randperm(len(train_input[0]))[:batch_size]
#     print(random_indeicies)
    train_batch_input = torch.LongTensor(train_batch_cat_value[random_indeicies])
    train_batch_gt = torch.FloatTensor(train_gt[random_indeicies])
#     print(train_batch_input)
#     print(train_batch_gt)
    
    # **************************************************
    out, attn_output_weights = model(train_batch_input)
    # **************************************************
    
#     print("Final_out:", out, "Gt:", train_batch_gt)
    loss_val = loss_func(out, torch.FloatTensor(train_batch_gt))
#     print("loss:", loss_val)
    total_loss += loss_val
    total_tokens += len(train_batch_input[0])
    loss_val.backward()
    optimizer.step()

    total_train_loss_by_token = total_loss / total_tokens

    model.eval()

    # Some validation here
    elapsed = time.time() - start
    if cur_step % 1000 == 0:
        print(f"Cur Step: {cur_step} Train Loss: {total_train_loss_by_token} elapsed: {elapsed}")
        val_category, val_intensity = valid_input
        val_category = torch.LongTensor(val_category)
        val_intensity = torch.FloatTensor(val_intensity)
        val_gt = torch.FloatTensor(valid_gt)
#         print(val_category, val_intensity)
        val_metrics = {
            "loss": loss_func,
            'auc': lambda pred, gt : roc_auc_score(gt.numpy(), pred.numpy())
            }
        val_result = validation(model, val_metrics, val_category, val_gt)
        print(val_result)
        log_result[cur_step] = val_result

Cur Step: 0 Train Loss: 0.07385271042585373 elapsed: 0.06293201446533203
torch.Size([1156, 1]) torch.Size([1156, 1])
{'loss': tensor(0.6666), 'auc': 0.48689806128830526}
Cur Step: 1000 Train Loss: 0.00902252271771431 elapsed: 0.0019049644470214844
torch.Size([1156, 1]) torch.Size([1156, 1])
{'loss': tensor(0.3373), 'auc': 0.5424640400250157}
Cur Step: 2000 Train Loss: 0.021527260541915894 elapsed: 0.0021643638610839844
torch.Size([1156, 1]) torch.Size([1156, 1])
{'loss': tensor(0.2435), 'auc': 0.7156920992286846}
Cur Step: 3000 Train Loss: 0.041874781250953674 elapsed: 0.0019969940185546875
torch.Size([1156, 1]) torch.Size([1156, 1])
{'loss': tensor(0.2577), 'auc': 0.6911663539712319}
Cur Step: 4000 Train Loss: 0.022433899343013763 elapsed: 0.002185821533203125
torch.Size([1156, 1]) torch.Size([1156, 1])
{'loss': tensor(0.2605), 'auc': 0.6391077756931415}
Cur Step: 5000 Train Loss: 0.01905936747789383 elapsed: 0.0020003318786621094
torch.Size([1156, 1]) torch.Size([1156, 1])
{'loss': t

In [22]:
log_result

{0: {'loss': tensor(0.6666), 'auc': 0.48689806128830526},
 1000: {'loss': tensor(0.3373), 'auc': 0.5424640400250157},
 2000: {'loss': tensor(0.2435), 'auc': 0.7156920992286846},
 3000: {'loss': tensor(0.2577), 'auc': 0.6911663539712319},
 4000: {'loss': tensor(0.2605), 'auc': 0.6391077756931415},
 5000: {'loss': tensor(0.2757), 'auc': 0.7156451949134875},
 6000: {'loss': tensor(0.2580), 'auc': 0.6999009797790285},
 7000: {'loss': tensor(0.3401), 'auc': 0.6377631853241611},
 8000: {'loss': tensor(0.2374), 'auc': 0.7105013550135503},
 9000: {'loss': tensor(0.2691), 'auc': 0.644063998332291},
 10000: {'loss': tensor(0.2740), 'auc': 0.699015009380863},
 11000: {'loss': tensor(0.2822), 'auc': 0.6940587867417135},
 12000: {'loss': tensor(0.2822), 'auc': 0.7246456118407338},
 13000: {'loss': tensor(0.3272), 'auc': 0.7035178236397749},
 14000: {'loss': tensor(0.3373), 'auc': 0.6955753595997499},
 15000: {'loss': tensor(0.3239), 'auc': 0.6411924119241194},
 16000: {'loss': tensor(0.3355), 'auc'

In [23]:
max(log_result.values(), key=lambda x : x['auc'])

{'loss': tensor(0.2822), 'auc': 0.7246456118407338}

In [24]:
min(log_result.values(), key=lambda x : x['loss'])

{'loss': tensor(0.2374), 'auc': 0.7105013550135503}

In [25]:
from model.utils import run_epoch

## CategoricalAttentionModel

In [26]:

len_seq = train_input[0].shape[1]
print("len_seq:", len_seq)


model = CategoricalAttentionModel(dict_category, d_model=d_model, n_head=n_head, len_seq=len_seq)
loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
batch_size = 128
n_steps = 20000
log_result_attention = dict()

model.train()
for cur_step in range(n_steps):
    """Train on single item of train data"""
    start = time.time()
    total_tokens = 0
    total_loss = 0

    model.train()
    
    train_batch_cat_value , train_batch_intensity_value = train_input
    random_indeicies = torch.randperm(len(train_input[0]))[:batch_size]
    train_batch_input = torch.LongTensor(train_batch_cat_value[random_indeicies])
    train_batch_intensity = torch.FloatTensor(train_batch_intensity_value[random_indeicies])
    train_batch_gt = torch.FloatTensor(train_gt[random_indeicies])
#     print(train_batch_input)
#     print(train_batch_gt)
    
    # **************************************************
    out, attn_output_weights = model([train_batch_input, train_batch_intensity])
    # **************************************************
    
#     print("Final_out:", out, "Gt:", train_batch_gt)
    loss_train = loss_func(out, torch.FloatTensor(train_batch_gt))
#     print("loss:", loss_val)
    total_loss += loss_val
    total_tokens += len(train_batch_input[0])
    
    # Back propagation
    loss_train.backward()
    optimizer.step()

    total_train_loss_by_token = total_loss / total_tokens

    model.eval()

    # Some validation here
    elapsed = time.time() - start
    if cur_step % 1000 == 0:
        print(f"Cur Step: {cur_step} Train Loss: {total_train_loss_by_token} elapsed: {elapsed}")
        val_category, val_intensity = valid_input
        val_category = torch.LongTensor(val_category)
        val_intensity = torch.FloatTensor(val_intensity)
        val_gt = torch.FloatTensor(valid_gt)
#         print(val_category, val_intensity)
        val_metrics = {
            "loss": loss_func,
            'auc': lambda pred, gt : roc_auc_score(gt.numpy(), pred.numpy())
            }
        val_result = validation(model, val_metrics, [val_category, val_intensity], val_gt)
        print(val_result)
        log_result_attention[cur_step] = val_result

len_seq: 9
Cur Step: 0 Train Loss: 0.016994865611195564 elapsed: 0.005062103271484375
torch.Size([1156, 1]) torch.Size([1156, 1])
{'loss': tensor(0.6933), 'auc': 0.3784083802376485}
Cur Step: 1000 Train Loss: 0.016994865611195564 elapsed: 0.0034208297729492188
torch.Size([1156, 1]) torch.Size([1156, 1])
{'loss': tensor(0.3404), 'auc': 0.6560245987075256}
Cur Step: 2000 Train Loss: 0.016994865611195564 elapsed: 0.0037000179290771484
torch.Size([1156, 1]) torch.Size([1156, 1])
{'loss': tensor(0.2606), 'auc': 0.7111528038357307}
Cur Step: 3000 Train Loss: 0.016994865611195564 elapsed: 0.0036361217498779297
torch.Size([1156, 1]) torch.Size([1156, 1])
{'loss': tensor(0.3234), 'auc': 0.7193089430894308}
Cur Step: 4000 Train Loss: 0.016994865611195564 elapsed: 0.003638029098510742
torch.Size([1156, 1]) torch.Size([1156, 1])
{'loss': tensor(0.2920), 'auc': 0.7397071086095477}
Cur Step: 5000 Train Loss: 0.016994865611195564 elapsed: 0.0034220218658447266
torch.Size([1156, 1]) torch.Size([1156, 

In [27]:
log_result_attention

{0: {'loss': tensor(0.6933), 'auc': 0.3784083802376485},
 1000: {'loss': tensor(0.3404), 'auc': 0.6560245987075256},
 2000: {'loss': tensor(0.2606), 'auc': 0.7111528038357307},
 3000: {'loss': tensor(0.3234), 'auc': 0.7193089430894308},
 4000: {'loss': tensor(0.2920), 'auc': 0.7397071086095477},
 5000: {'loss': tensor(0.3547), 'auc': 0.7681050656660412},
 6000: {'loss': tensor(0.3214), 'auc': 0.7632061705232437},
 7000: {'loss': tensor(0.3038), 'auc': 0.6353137377527621},
 8000: {'loss': tensor(0.3353), 'auc': 0.7478580362726704},
 9000: {'loss': tensor(0.2520), 'auc': 0.7463779445486762},
 10000: {'loss': tensor(0.2446), 'auc': 0.7567698561601001},
 11000: {'loss': tensor(0.2386), 'auc': 0.7705753595997497},
 12000: {'loss': tensor(0.2784), 'auc': 0.7505472170106317},
 13000: {'loss': tensor(0.2300), 'auc': 0.7513550135501357},
 14000: {'loss': tensor(0.2365), 'auc': 0.7522149259954137},
 15000: {'loss': tensor(0.2874), 'auc': 0.7418125912028353},
 16000: {'loss': tensor(0.3431), 'auc

In [28]:
max(log_result_attention.values(), key=lambda x : x['auc'])

{'loss': tensor(0.2386), 'auc': 0.7705753595997497}